In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "inputs"      )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Centers-Capacity-Tool


In [3]:
AnalysisAreas

'E:\\GitHub\\Centers-Capacity-Tool\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
sdf_AA

,FID,AreaName,Area,CenterLeve,QtrMileTS,SHAPE_Leng,SHAPE_Area,AnalysisAr,Analysis_1,SHAPE
0,0,Ballpark,WFRC MPO,City Center,No,1199.968937,46064.628342,Ballpark,City Center and No TOD,"{""rings"": [[[424765.7178999996, 4510247.8651],..."
1,1,Bountiful 200 West,WFRC MPO,City Center,No,1833.720189,113531.349978,Bountiful 200 West,City Center and No TOD,"{""rings"": [[[425450.29590000026, 4526560.92840..."
2,2,Bountiful Downtown,WFRC MPO,City Center,No,2723.929493,359566.725661,Bountiful Downtown,City Center and No TOD,"{""rings"": [[[425993.8905999996, 4526360.2699],..."
3,3,Butler Park Center,WFRC MPO,City Center,No,3064.698186,287001.994044,Butler Park Center,City Center and No TOD,"{""rings"": [[[431037.9064999996, 4496579.4782],..."
4,4,Central Antelope Center,WFRC MPO,City Center,No,3178.941458,442152.125447,Central Antelope Center,City Center and No TOD,"{""rings"": [[[412108.8102000002, 4549384.006200..."
...,...,...,...,...,...,...,...,...,...,...
419,419,West Jordan City Center,WFRC MPO,City Center,Yes,991.670030,25753.823922,West Jordan City Center,City Center and TOD,"{""rings"": [[[421533.7829, 4495458.125600001], ..."
420,420,West Jordan City Center,WFRC MPO,City Center,Yes,2732.365341,452288.309231,West Jordan City Center,City Center and TOD,"{""rings"": [[[420874.4562999997, 4494933.954700..."
421,421,West Valley/Taylorsville Redwood Road Center,WFRC MPO,City Center,Yes,2257.033028,329173.340637,West Valley/Taylorsville Redwood Road Center,City Center and TOD,"{""rings"": [[[420897.31319999974, 4507383.98650..."
422,422,West Valley/Taylorsville Redwood Road Center,WFRC MPO,City Center,Yes,2414.183003,359760.561272,West Valley/Taylorsville Redwood Road Center,City Center and TOD,"{""rings"": [[[420901.0235000001, 4505868.085100..."


In [5]:
#create map centered on Salt Lake
map_areas = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#classify land use
arcade_expression_aa = ("var v = $feature.Analysis_1;"
                        "if      (v=='City Center and No TOD'        ) { return 'cc'    ; }"
                        "else if (v=='City Center and TOD'           ) { return 'cc_tod'; }"
                        "else if (v=='Metropolitan Center and No TOD') { return 'mc'    ; }"
                        "else if (v=='Metropolitan Center and TOD'   ) { return 'mc_tod'; }"
                        "else if (v=='No Class Name'                 ) { return 'nc'    ; }"
                        "else if (v=='No Class Name and TOD'         ) { return 'nc_tod'; }"
                        "else if (v=='Urban Center and No TOD'       ) { return 'uc'    ; }"
                        "else if (v=='Urban Center and TOD'          ) { return 'uc_tod'; }")

#symbology for enrollment classes
uv_aa = [  
          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
        ]

#define sdf layer
sdf_AA.spatial.plot(map_widget = map_areas,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_areas.layout.height='500px'
map_areas.legend=True

#map title
display(Markdown('<h2><center>Analysis Areas</center></h2>'))

#display map
map_areas

<h2><center>Analysis Areas</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcels

In [6]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA)
sdf_ParcelsAA.groupby(['AreaID','ClassID'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum]})

,AreaID,ClassID,parcel_id,job_spaces,resunits,Acres
,,,size,sum,sum,sum
0,0,4,86.0,898.355010,3.995661,115.023128
1,1,4,223.0,188.947142,490.744795,48.003856
2,2,4,969.0,432.843096,1006.326474,144.966344
3,3,4,362.0,144.429158,515.468269,52.357758
4,4,4,192.0,31.266788,322.297674,33.314311
...,...,...,...,...,...,...
258,225,0,368.0,563.828947,279.720803,149.984264
259,226,0,619.0,1874.409413,686.732049,353.173643
260,226,1,224.0,1220.824477,138.439291,94.284615


In [7]:
sdf_ParcelsAA['county_id']

0         4.0
1         4.0
2         4.0
3         4.0
4         4.0
         ... 
137043    2.0
137044    2.0
137045    2.0
137046    2.0
137047    2.0
Name: county_id, Length: 137048, dtype: float64

# Land Use

In [8]:
display(sdf_ParcelsAA.columns)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"luType": "luTypeEx"})
display(sdf_ParcelsAA.columns)

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height', 'luType',
       'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt', 'resunits',
       'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co',
       'value_res', 'value_com', 'ParcelAcre', 'PieceAcres', 'PiecePorti',
       'Acres', 'SHAPE'],
      dtype='object')

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [9]:
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Residential'), 'luTypeEx'] = "Residential"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Commercial' ), 'luTypeEx'] = "Commercial"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & ((sdf_ParcelsAA['bldgtype'] != 'Residential') & (sdf_ParcelsAA['bldgtype'] != 'Commercial')), 'luTypeEx'] = "Developable"

In [23]:
#create map centered on Salt Lake
map_parcels_undevelopable = gis.map('Salt Lake')

countyfilter = 2


#Filter by County
sdf_ParcelsAA_lu = sdf_ParcelsAA[sdf_ParcelsAA['county_id']==countyfilter]
#sdf_ParcelsAA_lu

#classify land use
arcade_expression_lu = ("var v = $feature.luTypeEx;"
                        "if      (v=='Residential'  ) { return 'res'  ; }"
                        "else if (v=='Commercial'   ) { return 'com'  ; }"
                        "else if (v=='Developable'  ) { return 'dev'  ; }"
                        "else if (v=='Undevelopable') { return 'undev'; }"
                        "else                           { return 'other'  ; }")

#symbology for enrollment classes
uv_lu = [{"value":"res"  , "label":"Residential"  , "symbol":{"type":"esriSFS","color":[255,255,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"com"  , "label":"Commercial"   , "symbol":{"type":"esriSFS","color":[255,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"dev"  , "label":"No Building"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"undev", "label":"Undevelopable", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_lu.spatial.plot(map_widget = map_parcels_undevelopable,
                              renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                              unique_values=uv_lu,
                              arcade_expression=arcade_expression_lu,
                              default_symbol="" #don't include an 'other' category
                              )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_undevelopable,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_undevelopable.layout.height='500px'
map_parcels_undevelopable.legend=True

#map title
#display(Markdown('<h2><center>Land Use - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Existing Land Use</center></h2>'))

#display map
map_parcels_undevelopable

<h2><center>Existing Land Use</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Building Age

In [19]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [20]:
#set building age with base year 2019
sdf_ParcelsAA['BldAge'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['yearbuilt'] > 0)), 'BldAge'] = 2019 - sdf_ParcelsAA['yearbuilt']
display(sdf_ParcelsAA)
display(sdf_ParcelsAA.columns)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,sfvalue_re,sfvalue_co,value_res,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge
0,0,117.326952,835.775993,21,136,0,967,643515.0,643515.0,1.0,...,59.792523,0.0,137337.060741,0.0,0.206624,0.206525,0.999519,0.206525,"{""rings"": [[[413607.8234000001, 4462885.6642],...",17.0
1,1,109.679434,742.594505,21,136,0,968,643225.0,643225.0,1.0,...,53.048253,0.0,142099.021818,0.0,0.183590,0.183499,0.999506,0.183499,"{""rings"": [[[413635.0312000001, 4462961.596100...",20.0
2,2,109.751320,651.130977,21,136,0,969,697528.0,697528.0,1.0,...,66.142141,0.0,124055.611594,0.0,0.175349,0.160902,0.917608,0.160902,"{""rings"": [[[414229.6387, 4463098.214400001], ...",14.0
3,3,72.509390,144.724588,21,136,0,970,697559.0,697559.0,0.0,...,0.000000,0.0,0.000000,0.0,0.285292,0.035762,0.125353,0.035762,"{""rings"": [[[414135.5011, 4462989.6206], [4141...",0.0
4,4,41.838689,100.414511,21,136,0,973,722478.0,722478.0,0.0,...,0.000000,0.0,0.000000,0.0,0.024905,0.024813,0.996292,0.024813,"{""rings"": [[[414093.95249999966, 4462731.52800...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137043,137043,258.086311,4140.739077,64,164,0,765605,109643.0,806702.0,0.0,...,0.000000,0.0,0.000000,0.0,1.905318,1.023199,0.537023,1.023199,"{""rings"": [[[432707.19799999986, 4505834.80749...",0.0
137044,137044,431.493683,10458.588915,64,164,0,765627,109643.0,806724.0,0.0,...,0.000000,0.0,0.000000,0.0,2.586223,2.584374,0.999285,2.584374,"{""rings"": [[[432647.0683000004, 4505977.3521],...",0.0
137045,137045,408.848346,8959.388942,64,164,0,765628,109643.0,806725.0,0.0,...,0.000000,0.0,0.000000,0.0,4.293183,2.213913,0.515681,2.213913,"{""rings"": [[[432711.29700000025, 4505948.43930...",0.0
137046,137046,396.353415,9400.834422,64,164,0,765650,109643.0,806747.0,0.0,...,0.000000,0.0,0.000000,0.0,2.809431,2.322997,0.826857,2.322997,"{""rings"": [[[432647.65199999977, 4506078.8507]...",0.0


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [24]:
#create map centered on Salt Lake
map_parcels_bldgage = gis.map('Salt Lake')


countyfilter = 2

#Filter by County
sdf_ParcelsAA_bldg = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
#sdf_ParcelsAA_lu

#expression to classify enrollment
arcade_expression_bldgage = ("var v = $feature.BldAge;"
                             "if      (v==0) { return 'class1'; }"
                             "else if (v<40) { return 'class2'; }"
                             "else if (v<80) { return 'class3'; }"
                             "else           { return 'class4'; }")

#symbology for enrollment classes
uv_public = [{"value":"class1", "label":"No Building" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class2", "label":"Less than 40", "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class3", "label":"40 to 80"    , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class4", "label":"More than 80", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_bldg.spatial.plot(map_widget = map_parcels_bldgage,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_public,
                                arcade_expression=arcade_expression_bldgage,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_bldgage,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_bldgage.layout.height='500px'
map_parcels_bldgage.legend=True

#map title
#display(Markdown('<h2><center>Building Age by Parcel - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Building Age (Years) by Parcel</center></h2>'))

#display map
map_parcels_bldgage

<h2><center>Building Age (Years) by Parcel</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcel Value

In [25]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [26]:
#set building age with base year 2019
sdf_ParcelsAA['ValuePerAcre'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'ValuePerAcre'] = sdf_ParcelsAA['value_res'] / sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'ValuePerAcre'] = sdf_ParcelsAA['value_com'] / sdf_ParcelsAA['Acres']

#classify existing acreage by existing land use

sdf_ParcelsAA['acres_res'] = 0.0
sdf_ParcelsAA['acres_com'] = 0.0

sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'acres_res'] = sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'acres_com'] = sdf_ParcelsAA['Acres']

display(sdf_ParcelsAA)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge,ValuePerAcre,acres_res,acres_com
0,0,117.326952,835.775993,21,136,0,967,643515.0,643515.0,1.0,...,0.0,0.206624,0.206525,0.999519,0.206525,"{""rings"": [[[413607.8234000001, 4462885.6642],...",17.0,664990.823797,0.206525,0.0
1,1,109.679434,742.594505,21,136,0,968,643225.0,643225.0,1.0,...,0.0,0.183590,0.183499,0.999506,0.183499,"{""rings"": [[[413635.0312000001, 4462961.596100...",20.0,774385.394876,0.183499,0.0
2,2,109.751320,651.130977,21,136,0,969,697528.0,697528.0,1.0,...,0.0,0.175349,0.160902,0.917608,0.160902,"{""rings"": [[[414229.6387, 4463098.214400001], ...",14.0,771001.025107,0.160902,0.0
3,3,72.509390,144.724588,21,136,0,970,697559.0,697559.0,0.0,...,0.0,0.285292,0.035762,0.125353,0.035762,"{""rings"": [[[414135.5011, 4462989.6206], [4141...",0.0,0.000000,0.000000,0.0
4,4,41.838689,100.414511,21,136,0,973,722478.0,722478.0,0.0,...,0.0,0.024905,0.024813,0.996292,0.024813,"{""rings"": [[[414093.95249999966, 4462731.52800...",0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137043,137043,258.086311,4140.739077,64,164,0,765605,109643.0,806702.0,0.0,...,0.0,1.905318,1.023199,0.537023,1.023199,"{""rings"": [[[432707.19799999986, 4505834.80749...",0.0,0.000000,0.000000,0.0
137044,137044,431.493683,10458.588915,64,164,0,765627,109643.0,806724.0,0.0,...,0.0,2.586223,2.584374,0.999285,2.584374,"{""rings"": [[[432647.0683000004, 4505977.3521],...",0.0,0.000000,0.000000,0.0
137045,137045,408.848346,8959.388942,64,164,0,765628,109643.0,806725.0,0.0,...,0.0,4.293183,2.213913,0.515681,2.213913,"{""rings"": [[[432711.29700000025, 4505948.43930...",0.0,0.000000,0.000000,0.0
137046,137046,396.353415,9400.834422,64,164,0,765650,109643.0,806747.0,0.0,...,0.0,2.809431,2.322997,0.826857,2.322997,"{""rings"": [[[432647.65199999977, 4506078.8507]...",0.0,0.000000,0.000000,0.0


In [31]:
#create map centered on Salt Lake
map_parcels_areas_value = gis.map('Salt Lake')



countyfilter = 2

#Filter by County
sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
#sdf_ParcelsAA_lu

#expression to classify employment growth
#don't use value if 0
arcade_expression_value = ("var v = $feature.ValuePerAcre;"
                           "if      (v>0      && v<=100000) { return 'class4'; }"
                           "else if (v>100000 && v<=300000) { return 'class3'; }"
                           "else if (v>300000 && v<=500000) { return 'class2'; }"
                           "else if (v>500000             ) { return 'class1'; }")

#symbology for enrollment classes
uv_value = [{"value":"class1", "label":"More than 500k/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class2", "label":"$300k to $500k/acre" , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class3", "label":"$100k to $300k/acre" , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class4", "label":"Less than $100k/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_value,
                                arcade_expression=arcade_expression_value,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_areas_value.layout.height='500px'
map_parcels_areas_value.legend=True

#map title
display(Markdown('<h2><center>Value per Acre</center></h2>'))

#display map
map_parcels_areas_value

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


<h2><center>Value per Acre</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Development Categories

In [32]:
#ReDevelop, NewDevelop, NoDevelop

df_ClassParam = pd.read_csv(ClassParameters)
#display(df_ClassParam)

sdf_ParcelsAAwParam = pd.DataFrame.merge(sdf_ParcelsAA,df_ClassParam,on='ClassID',how='left')
#display(sdf_ParcelsAAwParam)

#undeveloped
sdf_ParcelsDevCat = sdf_ParcelsAAwParam.copy()
sdf_ParcelsDevCat['DevCategory'] = 'Remain'
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Developable'), 'DevCategory'] = "Developable"

#check if redevelopable - step 1: building age
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeRes']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategory'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeCom']) & (sdf_ParcelsDevCat['luTypeEx'] != 'Residential'), 'DevCategory'] = "Redevelopable"

#check if redevelopable - step 2: value
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreRes']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategory'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreCom']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Commercial' ), 'DevCategory'] = "Redevelopable"

#undevelopable acres
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Undevelopable'), 'DevCategory'] = "Undevelopable"

display(sdf_ParcelsDevCat[['yearbuilt','luTypeEx','ValuePerAcre','BldAge','DevCategory']])
sdf_ParcelsDevCat.columns

,yearbuilt,luTypeEx,ValuePerAcre,BldAge,DevCategory
0,2002.0,Residential,664990.823797,17.0,Remain
1,1999.0,Residential,774385.394876,20.0,Remain
2,2005.0,Residential,771001.025107,14.0,Remain
3,0.0,Developable,0.000000,0.0,Developable
4,0.0,Developable,0.000000,0.0,Developable
...,...,...,...,...,...
137043,0.0,Developable,0.000000,0.0,Developable
137044,0.0,Developable,0.000000,0.0,Developable
137045,0.0,Developable,0.000000,0.0,Developable
137046,0.0,Developable,0.000000,0.0,Developable


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge', 'ValuePerAcre', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevBldgAgeRes', 'RedevBldgAgeCom', 'GrowthRateRes', 'GrowthRateCom',
       'ClassOrder', 'DevCategory'],
      dtype='object')

In [33]:
display(df_ClassParam)
df_ClassParam.dtypes

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevBldgAgeRes,RedevBldgAgeCom,GrowthRateRes,GrowthRateCom,ClassOrder
0,3,Metropolitan Center and TOD,0.20,0.80,220,6,1200,600,0.1,500000,300000,80,40,1.05,1.05,1
1,2,Metropolitan Center and No TOD,0.25,0.75,110,3,1200,600,0.1,500000,300000,80,40,1.05,1.05,2
2,6,Urban Center and TOD,0.20,0.80,110,3,1200,600,0.1,500000,300000,80,40,1.05,1.05,3
3,5,Urban Center and No TOD,0.25,0.75,70,2,1200,600,0.1,500000,300000,80,40,1.05,1.05,4
4,1,City Center and TOD,0.50,0.50,70,2,1200,600,0.1,500000,300000,80,40,1.05,1.05,5
5,0,City Center and No TOD,0.50,0.50,35,1,1200,600,0.1,500000,300000,80,40,1.05,1.05,6
6,4,TOD Only,0.70,0.30,35,1,1200,600,0.1,500000,300000,80,40,1.05,1.05,7


ClassID                   int64
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA           int64
CapacityCom_FAR           int64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevBldgAgeRes           int64
RedevBldgAgeCom           int64
GrowthRateRes           float64
GrowthRateCom           float64
ClassOrder                int64
dtype: object

In [34]:
countyfilter = 2

#Filter by County
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat.copy()
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat_filter[sdf_ParcelsDevCat_filter['county_id']==countyfilter]

#create map centered on Salt Lake
map_parcels_dev = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_dev = ("var v = $feature.DevCategory;"
                             "if      (v=='Undevelopable') { return 'class2'; }"
                             "else if (v=='Remain'       ) { return 'class2'; }"
                             "else if (v=='Redevelopable') { return 'class3'; }"
                             "else if (v=='Developable'  ) { return 'class4'; }"
                        )

#symbology for enrollment classes
uv_dev = [{"value":"class4", "label":"Developable"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class3", "label":"Redevelopable", "symbol":{"type":"esriSFS","color":[128,  0,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class2", "label":"To Remain"    , "symbol":{"type":"esriSFS","color":[180,180,180,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsDevCat_filter.spatial.plot(map_widget = map_parcels_dev,
                                      renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                      unique_values=uv_dev,
                                      arcade_expression=arcade_expression_dev,
                                      default_symbol="" #don't include an 'other' category
                                      )
#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_dev,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_dev.layout.height='500px'
map_parcels_dev.legend=True

#map title
display(Markdown('<h2><center>Development Category - County ' + str(countyfilter) + '</center></h2>'))

#display map
map_parcels_dev

<h2><center>Development Category - County 2</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [35]:
#aggregate parcels into analysis areas / classes
df_AADevCat = sdf_ParcelsDevCat.groupby(['AreaID','ClassID','DevCategory'],as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum], "sf_res":[np.sum], "sf_com":[np.sum], "acres_res":[np.sum], "acres_com":[np.sum]})
df_AADevCat.columns = df_AADevCat.columns.droplevel(1)
display(df_AADevCat)


df_AADevCat.to_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'),index=False)


,AreaID,ClassID,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,4,Developable,52.0,0.000000,0.000000,61.545706,0.000000,0.000000,0.000000,0.000000
1,0,4,Remain,22.0,281.805549,1.997830,45.557999,2643.416033,200289.295120,5.283979,40.274020
2,0,4,Undevelopable,12.0,616.549461,1.997831,7.919424,2878.874782,601129.519419,0.000000,0.000000
3,1,4,Developable,42.0,0.000000,0.000000,5.587282,0.000000,0.000000,0.000000,0.000000
4,1,4,Redevelopable,103.0,89.807474,151.710205,21.659572,168460.165094,49316.995519,13.171002,8.488570
...,...,...,...,...,...,...,...,...,...,...,...
969,227,0,Undevelopable,54.0,122.607502,10.537570,88.129503,12496.631974,44577.315860,0.000000,0.000000
970,227,1,Developable,76.0,0.000000,0.000000,21.576751,0.000000,0.000000,0.000000,0.000000
971,227,1,Redevelopable,203.0,1079.638350,168.278154,83.627682,244576.070866,535108.183782,36.225087,47.402595
972,227,1,Remain,252.0,1419.898355,219.235224,58.935571,298303.199946,813247.523389,19.297788,39.637783
